In [40]:
# ライブラリのインストール
%pip install transformers fugashi ipadic sentencepiece

from transformers import BertJapaneseTokenizer, BertModel
import torch
import numpy as np
import pandas as pd
import os


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [41]:

class SentenceBertJapanese:
    def __init__(self, model_name_or_path, device=None):
        self.tokenizer = BertJapaneseTokenizer.from_pretrained(model_name_or_path)
        self.model = BertModel.from_pretrained(model_name_or_path)
        self.model.eval()

        if device is None:
            device = "cuda" if torch.cuda.is_available() else "cpu"
        self.device = torch.device(device)
        self.model.to(device)

    def _mean_pooling(self, model_output, attention_mask):
        token_embeddings = model_output[0]  # モデル出力の最初の要素がトークン埋め込みを含む
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

    @torch.no_grad()
    def encode(self, sentences, batch_size=8):
        all_embeddings = []
        for batch_idx in range(0, len(sentences), batch_size):
            batch = sentences[batch_idx:batch_idx + batch_size]

            encoded_input = self.tokenizer.batch_encode_plus(batch, padding="max_length", max_length=512,
                                           truncation=True, return_tensors="pt").to(self.device)
            model_output = self.model(**encoded_input)
            sentence_embeddings = self._mean_pooling(model_output, encoded_input["attention_mask"]).to('cpu')

            all_embeddings.extend(sentence_embeddings)

        return torch.stack(all_embeddings)

def cos_sim(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))


# テキストファイルを読み込み、各文をリストに格納
def read_sentences_from_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        sentences = f.read().splitlines()
    return sentences


In [42]:

# モデルの初期化
model1 = SentenceBertJapanese("sonoisa/sentence-bert-base-ja-mean-tokens-v2")
model2 = SentenceBertJapanese("tohoku-nlp/bert-base-japanese")
model3 = SentenceBertJapanese("tohoku-nlp/bert-base-japanese-v3")
model4 = SentenceBertJapanese("tohoku-nlp/bert-base-japanese-char-v3")
model5 = SentenceBertJapanese("tohoku-nlp/bert-large-japanese-v2")
model6 = SentenceBertJapanese("tohoku-nlp/bert-large-japanese-char-v2")


c:\WorkSpace\Practice\env2\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at tohoku-nlp/bert-base-japanese were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of

In [43]:
for _,_, files in os.walk("E:\実験\テキスト/test"):
    print(files)
    for i in range(0, len(files)):
        file = str(files[i])
        num = file[-6:-4]
        # print(num)

        file_path = "E:\実験\テキスト/test/文章" + str(num) + ".txt"
        save_path = "C:\WorkSpace\Practice\コサイン類似度/avecossim_result" + str(num) + ".csv"
        sentences = read_sentences_from_file(file_path)

        # 文をエンコード
        embeddings1 = model1.encode(sentences)
        embeddings2 = model2.encode(sentences)
        embeddings3 = model3.encode(sentences)
        embeddings4 = model4.encode(sentences)
        embeddings5 = model5.encode(sentences)
        embeddings6 = model6.encode(sentences)

        total_vec1=0
        ave_vec1 =0
        for j in range(0, len(embeddings1)):
            total_vec1 += embeddings1[j] 
            ave_vec1 = total_vec1 / len(embeddings1)

        total_vec2=0
        ave_vec2 =0
        for j in range(0, len(embeddings2)):
            total_vec2 += embeddings2[j] 
            ave_vec2 = total_vec2 / len(embeddings2)

        total_vec3=0
        ave_vec3 =0
        for j in range(0, len(embeddings3)):
            total_vec3 += embeddings3[j] 
            ave_vec3 = total_vec3 / len(embeddings3)

        total_vec4=0
        ave_vec4 =0
        for j in range(0, len(embeddings4)):
            total_vec4 += embeddings4[j] 
            ave_vec4 = total_vec4 / len(embeddings4)

        total_vec5=0
        ave_vec5 =0
        for j in range(0, len(embeddings5)):
            total_vec5 += embeddings5[j] 
            ave_vec5 = total_vec5 / len(embeddings5)

        total_vec6=0
        ave_vec6 =0
        for j in range(0, len(embeddings6)):
            total_vec6 += embeddings6[j] 
            ave_vec6 = total_vec6 / len(embeddings6)


        avecossim_result1 = []
        avecossim_result2 = []
        avecossim_result3 = []
        avecossim_result4 = []
        avecossim_result5 = []
        avecossim_result6 = []

        for k in range(0, len(sentences)):
            avecossim_result1.append(cos_sim(ave_vec1.numpy(), embeddings1[k].numpy()))
            avecossim_result2.append(cos_sim(ave_vec2.numpy(), embeddings2[k].numpy()))
            avecossim_result3.append(cos_sim(ave_vec3.numpy(), embeddings3[k].numpy()))
            avecossim_result4.append(cos_sim(ave_vec4.numpy(), embeddings4[k].numpy()))
            avecossim_result5.append(cos_sim(ave_vec5.numpy(), embeddings5[k].numpy()))
            avecossim_result6.append(cos_sim(ave_vec6.numpy(), embeddings6[k].numpy()))


        df = pd.DataFrame({
            "sonoisa/sentence-bert-base-ja-mean-tokens-v2" : avecossim_result1,
            "tohoku-nlp/bert-base-japanese" : avecossim_result2,
            "tohoku-nlp/bert-base-japanese-v3" : avecossim_result3,
            "tohoku-nlp/bert-base-japanese-char-v3" : avecossim_result4,
            "tohoku-nlp/bert-large-japanese-v2" : avecossim_result5,
            "tohoku-nlp/bert-large-japanese-char-v2" : avecossim_result6
        })

        df.to_csv(save_path, mode="a", header=True, index=False)

        print(f"{num}を記録しました。")

['文章22.txt', '文章23.txt', '文章24.txt', '文章25.txt', '文章01.txt', '文章02.txt', '文章04.txt', '文章06.txt', '文章07.txt', '文章09.txt', '文章10.txt', '文章11.txt', '文章14.txt', '文章15.txt', '文章17.txt', '文章18.txt', '文章19.txt', '文章21.txt', '文章27.txt', '文章29.txt', '文章33.txt', '文章36.txt', '文章42.txt', '文章45.txt', '文章46.txt', '文章26.txt']
22を記録しました。
23を記録しました。
24を記録しました。
25を記録しました。
01を記録しました。
02を記録しました。
04を記録しました。
06を記録しました。
07を記録しました。
09を記録しました。
10を記録しました。
11を記録しました。
14を記録しました。
15を記録しました。
17を記録しました。
18を記録しました。
19を記録しました。
21を記録しました。
27を記録しました。
29を記録しました。
33を記録しました。
36を記録しました。
42を記録しました。
45を記録しました。
46を記録しました。
26を記録しました。


In [44]:
for i in range(1, len(sentences)):
    print(sentences[i])

彼らが文明の証とみなしていたものは、農業による食料生産の開始、製鉄をはじめとする治金技術の発達、さまざまな技術的発明、集権的な集団組織の確立、そして文字の発明などである。
これらの要素のうち、文字の発明は特定の地域で起こったものと昔から考えられてきた。
たとえばイスラム圏の拡大や、ヨーロッパ人の植民地支配がはじまるまで、オーストラリア、太平洋諸島、赤道アフリカの人びとは、文字というものを持っていなかった。
南北アメリカ大陸においても、中米の一部のごく狭い地域を除いて、文字を持つようになったのはヨーロッパ人の植民地支配が確立されてからのことである。
文字の使用地域が世界的に極端に限定されていたことから、自分たちを文明人と思い込んでいた人びとは、文字を使える能力こそ、「野蛮人」や「未開人」に対する「文明人」の優越性を示すもっとも顕著なちがいだと考えていた。
文字は、はるか遠く離れた世界についての知識をわれわれに教えてくれる。
昔の出来事をわれわれに伝えてくれる。
また、大量の知識を正確に伝達することができる。
もちろん、インカ人たちのように、文字を持たずして大帝国を治めることができた人びともいた。
フン族相手に大敗したローマ軍のように、「文明人」が「野蛮人」に常に勝利できたわけではない。
しかし、文字は知識をもたらす。
ヨーロッパ人が南北アメリカ大陸、シベリア、そしてオーストラリアを征服したことは、文字が知識をもたらし、知識が力をもたらすことを如実に示している。
文字は、武器、病原菌、そして集権化された政治機構などとともに、近代ヨーロッパ人の海外遠征に付随して、世界のさまざまな場所に伝達していった。
船隊を派遣した君主や大商人は、文書をしたため、命令や指示を伝えた。
そうした船隊は、先遣隊の残した海図や航海誌を手がかりに進路をとった。
遠征隊の記録は、莫大な富や肥沃な土地について語り、さらなる遠征の旅へと人びとをかりたてた。
これらの記録は、あとにつづく者に、どのような環境が待ち受けているかを伝えた。
征服者が現地に誕生させた帝国の統治は、文字を使っておこなわれた。
もちろん、文字を持たなかった社会でも、さまざまな手段で情報は伝達されていた。
しかし、文字は、情報をはるかに容易に、はるかに正確に、はるかに説得力のあるかたちで伝達できるようにした。
